# Typehints with semantic menaings:

I desire typehints that specify semantic meanings in addition to their type, like for Dict and Tuples.

I currently have a python dictionary named `sessions` of type `Dict[str, Tuple[Path, datetime]]`, an example entry would be: 
```python 
sessions['kdiba_gor01_one_2006-6-08_14-26-15']
>> {'ripple_time_bin_marginals_df': (Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-24_0745AM-kdiba_gor01_one_2006-6-08_14-26-15-(ripple_time_bin_marginals_df).csv'),
  datetime.datetime(2024, 1, 24, 7, 45)),
 'ripple_marginals_df': (Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-24_0745AM-kdiba_gor01_one_2006-6-08_14-26-15-(ripple_marginals_df).csv'),
  datetime.datetime(2024, 1, 24, 7, 45)),
 'laps_time_bin_marginals_df': (Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-24_0745AM-kdiba_gor01_one_2006-6-08_14-26-15-(laps_time_bin_marginals_df).csv'),
  datetime.datetime(2024, 1, 24, 7, 45)),
 'laps_marginals_df': (Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-24_0745AM-kdiba_gor01_one_2006-6-08_14-26-15-(laps_marginals_df).csv'),
  datetime.datetime(2024, 1, 24, 7, 45))}
```
I'd like to be able to specify the semantic menaing of the dictionary elements in the type definition, for reference later (documentation) and potentially (down the road) to enforce consistency by implementing checks on certain properties. I'm hoping for something like: `Dict[session_id:str, Tuple[file_path:Path, parsed_datetime:datetime]]` instead of `Dict[str, Tuple[Path, datetime]]`.

Dict[str, Tuple[Path, datetime]]
Dict[session_id:str, Tuple[file_path:Path, parsed_datetime:datetime]]



## Option 1: `typing.NewType` to define aliases

In [ ]:
from typing import Dict, Tuple, NewType
from datetime import datetime
from pathlib import Path

# defining type aliases
session_id = NewType('session_id', str)
file_path = NewType('file_path', Path)
parsed_datetime = NewType('parsed_datetime', datetime)

# now you can define your dictionary in terms of these new types
sessions: Dict[session_id, Tuple[file_path, parsed_datetime]]

# Option 2: Inline `attrs` classes

In [ ]:
import attrs
from collections import namedtuple
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import Zscorer
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import LongShortStatsTuple

# dict(zip(['long_stats_z_scorer', 'short_stats_z_scorer', 'long_short_z_diff', 'long_short_naive_z_diff', 'is_forward_replay'], [Zscorer, Zscorer, float, float, bool]))
LongShortStatsOutput = attrs.make_class('LongShortStatsOutput', ['long_stats_z_scorer', 'short_stats_z_scorer', 'long_short_z_diff', 'long_short_naive_z_diff', 'is_forward_replay'], slots=False, order=True)  # , bases=[LongShortStatsTuple]

# attrs.define(


In [ ]:
# Creating class using make_class
SessionEntry = attrs.make_class("SessionEntry", ["file_path", "parsed_datetime"], frozen=True, slots=True)

# Assign types to attributes
attrs.set_run_validators(SessionEntry, {
    "file_path": attrs.validators.instance_of(Path),
    "parsed_datetime": attrs.validators.instance_of(datetime)
})

entry = SessionEntry(file_path=Path('/path/to/file'), parsed_datetime=datetime.now())



# 2024-08-12 - typing.Annotated

In [1]:
from typing import Annotated
from typing import Dict, List, Tuple, Optional, Callable, Union, Any, NewType, TypeVar
from typing_extensions import TypeAlias
import numpy as np
from numpy.typing import NDArray

Shape = Tuple[int, ...]

def validate_shape(array: NDArray, shape: Shape) -> NDArray:
    if array.shape != shape:
        raise ValueError(f"Expected array with shape {shape}, but got {array.shape}.")
    return array

ArrayWithShape = Annotated[NDArray, validate_shape]



In [2]:
n_predicted_t_bins = TypeVar('n_predicted_t_bins')
n_x_bins = TypeVar('n_x_bins')
n_t_bins = TypeVar('n_t_bins')

# DecoderListDict: TypeAlias = Dict[str, List[T]]

ArrayWithShape = Annotated[NDArray[Tuple[int, int, int]], (n_predicted_t_bins, n_x_bins, n_t_bins)]


In [ ]:

# Example usage
Array5x24 = Annotated[NDArray[Tuple[int, int]], (5, 24)]

# To validate at runtime
def create_array(array: NDArray, shape: Shape) -> ArrayWithShape:
    return validate_shape(array, shape)

# Usage
# array: ArrayWithShape = np.random.rand(5, 24)
array: ArrayWithShape = np.random.rand(5, 5)
array

In [ ]:
validated_array = create_array(array, (5, 24))  # This will pass
validated_array

In [ ]:
array = np.random.rand(5, 2)
validated_array = create_array(array, (5, 24))  # This will pass
validated_array


In [ ]:
class SizedArray(np.ndarray):
    def __new__(cls, input_array, *args, **kwargs):
        obj = np.asarray(input_array).view(cls)
        # Additional initialization can be done here if needed
        return obj
    
    def custom_method(self):
        # Define your custom method here
        return np.sum(self)  # Example: sum of the array elements

    def __array_finalize__(self, obj):
        # This method is called automatically to finalize the array object.
        if obj is None: return
        # Copy any attributes from `obj` to `self` if needed
        